# Mediana dla obrazu kolorowego

Idea filtracji medianowej jest dość prosta dla obrazów w odcieniach szarości.
Dla obrazów kolorowych trudniej jest określić kryterium wg. którego szeregowane będą wartości, z których wyznaczana będzie mediana.

Jedną z możliwości wykonania filtracji medianowej dla obrazów kolorowych (na podstawie *The Image Processing Handbook*, J. Russ) jest wykorzystanie następującej definicji mediany:
``mediana to ten piksel z otoczenia, którego odległość do innych pikseli z otoczenia jest najmniejsza''.
Jako miarę odległości wykorzystujemy pierwiastek z sumy kwadratów różnic poszczególnych składowych R,G,B.
Zatem odległość między dwoma pikselami wyraża się wzorem:
\begin{equation}
dRGB = \sqrt{(R_1-R_2)^2+(G_1-G_2)^2+(B_1-B_2)^2}
\end{equation}

Warto zwrócić uwagę, że istnieje wiele możliwości zdefiniowania porównywania wielkości wektorowych (jeden piksel to wektor o trzech składowych).
Można zamiast odległości wykorzystać kąt albo połączyć oba parametry.
Ponadto istnieje możliwość dodania do wektora dodatkowych składowych - tak aby lepiej opisać piksel.

Celem zadania jest implementacja opisanego algorytmu.

1. Wczytaj obraz *lenaRGBSzum.png* (dostępny na git).
2. Zdefiniuj rozmiar okna.
3. Wykonaj pętle po pikselach, dla których okno jest zdefiniowane (pomiń brzeg obrazu).
4. Dla każdego piksela pobierz okno o właściwym rozmiarze.
5. Wykonaj pętle po oknie, wewnątrz której obliczona zostanie suma odległości.
    - Obliczanie różnicy: `window - window[rowWin, colWin]`.
    - Obliczanie kwadratów: `np.square`.
    - Obliczanie pierwiastka: `np.sqrt`.
    - Obliczanie sumy metodą `.sum`.
6. Po obliczeniu macierzy odległości wyznacz argument elementu minimalnego.
Wykorzystaj funkcję `np.argmin`.
Argument funkcji zostanie spłaszczony, jeśli ma więcej niż jeden wymiar.
Aby przekonwertować spłaszczony indeks na indeks macierzy wykorzystaj funkcję `np.unravel_index`.
7. Przypisz odpowiedni wektor wartości do piksela obrazu wynikowego.
8. Wyświetl obraz oryginalny i przefiltrowany.
9. Przeprowadź dwa eksperymenty - dla obrazu _lenaRGB_ oraz _lenaRGBszum_.

In [1]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/06_Context/'

fileNames = ["lenaRGB.png", "lenaRGBSzum.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

In [2]:
def color_median_filter(image, kernel_size):
    # Ensure kernel size is odd to have a proper window

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.int32)
    if kernel_size % 2 == 0:
        raise ValueError("Kernel size must be odd")

    pad_size = kernel_size // 2
    padded_image = cv2.copyMakeBorder(image, pad_size, pad_size, pad_size, pad_size, cv2.BORDER_REFLECT)

    filtered_image = np.zeros_like(image)

    for i in range(pad_size, padded_image.shape[0] - pad_size):
        for j in range(pad_size, padded_image.shape[1] - pad_size):
            # Get the window of pixels around the current pixel
            window = padded_image[i - pad_size:i + pad_size + 1, j - pad_size:j + pad_size + 1]
            temp = np.zeros((kernel_size, kernel_size))

            # Flatten the window to apply vectorized computation and calculate distances
            for wi in range(kernel_size):
                    for wj in range(kernel_size):
                        distances = np.square(window - window[wi, wj])
                        distances = np.sum(distances, axis=-1)
                        temp[wi, wj] = np.sqrt(distances).sum()

            # Find the index with the smallest sum of distances
            min_idx = np.argmin(temp)

            # Convert the flattened index back to row and column indices and get the median pixel value
            min_idx = np.unravel_index(min_idx, window.shape[:2])

            # Assign the median pixel to the output image
            filtered_image[i - pad_size, j - pad_size] = window[min_idx]

    return filtered_image

In [4]:
IRGB = cv2.imread("lenaRGB.png")
IRGBS = cv2.imread("lenaRGBSzum.png")

f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 15))

ax1.imshow(cv2.cvtColor(IRGB, cv2.COLOR_BGR2RGB).astype(np.int32))
ax1.set_title("Original RGB")

ax2.imshow(color_median_filter(IRGB, 3))
ax2.set_title("Filtered RGB")

ax3.imshow(cv2.cvtColor(IRGBS, cv2.COLOR_BGR2RGB).astype(np.int32))
ax3.set_title("Original RGB with noise")

ax4.imshow(color_median_filter(IRGBS, 3))
ax4.set_title("Filtered RGB with noise")

Output hidden; open in https://colab.research.google.com to view.